# Xarray

Get exactly the data you need, then aggregate or iteratively operate over that data.

## What is Xarray?

Xarray provides a container for accessing metadata right alongside data. 

![dataset-diagram](images/dataset-diagram.png)

Metadata can include:
 - arrays of index values (coordinates 
 - units
 - projection info
 - origin of data and processing history
 


In [ ]:
import xarray as xr

In [ ]:
ds = xr.open_dataset('./tutorial-data/sst/NOAA_NCDC_ERSST_v3b_SST-1960.nc')
ds

## Subset data

Get all the `sst` values for one particular time step. 

In [ ]:
ds.sel(time="1960-12-15").sst

In [ ]:
ds.sel(time="1960-12-15").sst.plot()

In [ ]:
ds.sel(lat=slice(0, 60), lon=slice(230, 300), time="1960-12-15").sst.plot()

## Aggregate

In [ ]:
ds.sst.mean(dim='lon').plot(x="time", y="lat")

In [ ]:
ds.sst.std(dim=('time', 'lon')).plot()

# Dask

Work with data that doesn't fit in memory and finish computations more quickly.

## What is Dask?

Dask delays, optimizes, parallizes and distributes computations. Instead of computing as soon as you execute a line of code, Dask custructs a task graph. That task graph keeps getting added to until you trigger computation.

Xarray uses Dask internally to store references to the data so that it can be accessed chunk by chunk. 

![Dask array](images/dask-array-black-text.png)

In [ ]:
ds_all = xr.open_mfdataset('./tutorial-data/sst/*nc', combine='by_coords')
ds_all

## Subset data

Get all the `sst` values for one particular time step. 

In [ ]:
ds_all.sel(time="2016-12-15").sst

If you access the data within this xarray object, you'll see that it is a Dask array.

In [ ]:
ds_all.sel(time="2016-12-15").sst.data

## Groupby

In [ ]:
sst_clim = ds_all.sst.groupby('time.month').mean(dim='time')
sst_clim

In [ ]:
(sst_clim[7] - sst_clim[0]).plot()

## Resample

In [ ]:
sst_ts = ds_all.sst.sel(lon=275, lat=30, method="nearest")
sst_ts.resample(time='A').mean(dim='time')

In [ ]:
sst_ts.resample(time='A').mean(dim='time').plot()

In [ ]:
# note that this is equivalent to doing a groupby on year.
sst_ts.groupby('time.year').mean(dim='time')

## Distribute computations

So far we have been focussing on how regular xarray operations chain together to create a task graph. This is the middle part of the image below:

![Dask overview](images/dask-overview.png)

Until you trigger computation, Dask just keeps adding layers to the task graph. But once you call `.plot` or `.persist` or `.compute`, that task graph gets sent to the scheduler for execution.

In [ ]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()  # in practice this might be a kubernetes cluster, or a slurm cluster
client = Client(cluster)
client

In the notebook, you interact with the `client`. The `client` sends task graphs to the `cluster`. Within the `cluster` there is one `scheduler` and many`workers`. The scheduler is in charge of optimizing the task grpha, assigning tasks to partular workers and collecting results.

![Distributed overview](images/distributed-overview.png)

### Diagnostics Dashboard

If you have a `client`, then you can see what Dask is up to by checking out the dashboard. 

In [ ]:
sst_ts.groupby('time.year').mean(dim='time').compute()

# Best Practices

- Use cloud-optimized file formats
- Subset data as early as possible in computation
- Consider chunksize and if you need to rechunk, use `persist` or save a different representation. 
    - Consider using dedicated tooling such as [kerchunk](https://github.com/fsspec/kerchunk).
- If you have multiple outputs, use `dask.compute` to combine multiple task graphs.

# Resources

This notebook borrows heavily from the Pangeo Xarray Tutorial, so that one will look familiar, but the original contains more detail and explanation. 

- [Pangeo Xarray Tutorial](http://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/xarray.html)
- [Pangeo Dask Tutorial](http://gallery.pangeo.io/repos/pangeo-data/pangeo-tutorial-gallery/dask.html)